# This notebook is initial modeling for the Honk4Honda data challenge (unsupervised learning)

## Import reduced dataset

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt

rd = pd.read_csv('./data/reduced_data.csv')
rd.head()

,price,year,manufacturer,make,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,weather,location
0,11900,2010.0,dodge,challenger se,good,6,gas,43600.0,clean,automatic,rwd,NaN,coupe,red,38.0,"marshall, SD"
1,17550,2008.0,ford,f-150,NaN,NaN,gas,NaN,clean,automatic,NaN,NaN,NaN,NaN,38.0,"marshall, SD"
2,2800,2004.0,ford,taurus,good,6,gas,168591.0,clean,automatic,fwd,full-size,sedan,grey,47.0,"marshall, IA"
3,9900,2007.0,gmc,yukon,good,8,gas,169000.0,clean,automatic,4wd,NaN,NaN,NaN,38.0,"marshall, SD"
4,12500,2015.0,jeep,patriot high altitude,like new,4,gas,39500.0,rebuilt,automatic,4wd,sub-compact,SUV,grey,38.0,"marshall, SD"


In [3]:
len(rd)

1526179

In [5]:
# Check for any missing data
print("Table size -", end=' ')
print(rd.shape)

print("Checking for missing values..")
# Number of missing values in each column of training data
missing_val_count_by_column = (rd.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

# Get names of columns with missing values
missing_cols = [col for col in rd.columns
                     if rd[col].isnull().any()]

Table size - (1526179, 16)
Checking for missing values..
make             67936
condition       624187
cylinders       597331
fuel              8710
odometer        475926
title_status      1250
transmission      7294
drive           568182
size            991001
type            604737
paint_color     599400
dtype: int64


# Try to fill in some or all of the missing values

In [10]:
# For make, condition, size, type, paint_color I can fill in as unknown
rd = rd.fillna(value={'make':'unknown','condition':'unknown','size':'unknown','type':'unknown','paint_color':'unknown'})
rd.head()

,price,year,manufacturer,make,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,weather,location
0,11900,2010.0,dodge,challenger se,good,6,gas,43600.0,clean,automatic,rwd,unknown,coupe,red,38.0,"marshall, SD"
1,17550,2008.0,ford,f-150,unknown,NaN,gas,NaN,clean,automatic,NaN,unknown,unknown,unknown,38.0,"marshall, SD"
2,2800,2004.0,ford,taurus,good,6,gas,168591.0,clean,automatic,fwd,full-size,sedan,grey,47.0,"marshall, IA"
3,9900,2007.0,gmc,yukon,good,8,gas,169000.0,clean,automatic,4wd,unknown,unknown,unknown,38.0,"marshall, SD"
4,12500,2015.0,jeep,patriot high altitude,like new,4,gas,39500.0,rebuilt,automatic,4wd,sub-compact,SUV,grey,38.0,"marshall, SD"


In [11]:
# For cylinders the majority are 4,6,8. 6 is a good estimate
rd['cylinders'].value_counts()

6      327942
8      291478
4      260269
oth     33101
5        8799
10       5623
3        1038
12        598
Name: cylinders, dtype: int64

In [12]:
# For fuel ~ 90% gas so that's a good estimate
rd['fuel'].value_counts()

gas         1368274
diesel        98801
other         39256
hybrid        10065
electric       1073
Name: fuel, dtype: int64

In [14]:
# For odometer take the mean
round(rd['odometer'].mean(),1)

113193.6

In [15]:
# For title status ~ 90% clean so that's a good estimate. Also very few of these are missing so no need for separate category
rd['title_status'].value_counts()

clean         1418897
rebuilt         44242
salvage         29487
lien            20001
missing          8539
parts only       3763
Name: title_status, dtype: int64

In [16]:
# For transmission ~ 90% automatic so that's a good estimate. Also very few of these are missing so no need for separate category
rd['transmission'].value_counts()

automatic    1324632
manual        166276
other          27977
Name: transmission, dtype: int64

In [17]:
# For drive fairly well balanced so let's give this unknown
rd['drive'].value_counts()

4wd    408327
fwd    334534
rwd    215136
Name: drive, dtype: int64

In [20]:
# Let's fill these columns
rd = rd.fillna(value={'cylinders':'6','fuel':'gas','odometer':113193.6,'title_status':'clean','transmission':'automatic','drive':'unknown'})
rd.head()

,price,year,manufacturer,make,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,weather,location
0,11900,2010.0,dodge,challenger se,good,6,gas,43600.0,clean,automatic,rwd,unknown,coupe,red,38.0,"marshall, SD"
1,17550,2008.0,ford,f-150,unknown,6,gas,113193.6,clean,automatic,unknown,unknown,unknown,unknown,38.0,"marshall, SD"
2,2800,2004.0,ford,taurus,good,6,gas,168591.0,clean,automatic,fwd,full-size,sedan,grey,47.0,"marshall, IA"
3,9900,2007.0,gmc,yukon,good,8,gas,169000.0,clean,automatic,4wd,unknown,unknown,unknown,38.0,"marshall, SD"
4,12500,2015.0,jeep,patriot high altitude,like new,4,gas,39500.0,rebuilt,automatic,4wd,sub-compact,SUV,grey,38.0,"marshall, SD"


In [22]:
# recheck for any missing data
print("Table size -", end=' ')
print(rd.shape)

print("Checking for missing values..")
# Number of missing values in each column of training data
missing_val_count_by_column = (rd.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

# Get names of columns with missing values
missing_cols = [col for col in rd.columns
                     if rd[col].isnull().any()]

Table size - (1526179, 16)
Checking for missing values..
Series([], dtype: int64)


# Missing values dealt with, now I need to invert this dataframe somehow to make it a function of location

## But first, decide the locations to keep

In [40]:
loc_list = rd['location'].value_counts()

In [45]:
# 725 locations which have > 100 counts. Use these to cluster. Remove the other locations
len(loc_list[loc_list>100])

725

In [52]:
loc_list2 = (loc_list[loc_list>100]).index.tolist()

In [87]:
# Important if I want to cluster based on this
'omaha, NE' in loc_list2

True

In [82]:
# This operation removes around 30k cars
rd_loc=rd[rd['location'].isin(loc_list2)]
rd_loc.head()

,price,year,manufacturer,make,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,weather,location
0,11900,2010.0,dodge,challenger se,good,6,gas,43600.0,clean,automatic,rwd,unknown,coupe,red,38.0,"marshall, SD"
1,17550,2008.0,ford,f-150,unknown,6,gas,113193.6,clean,automatic,unknown,unknown,unknown,unknown,38.0,"marshall, SD"
3,9900,2007.0,gmc,yukon,good,8,gas,169000.0,clean,automatic,4wd,unknown,unknown,unknown,38.0,"marshall, SD"
4,12500,2015.0,jeep,patriot high altitude,like new,4,gas,39500.0,rebuilt,automatic,4wd,sub-compact,SUV,grey,38.0,"marshall, SD"
6,1250,2004.0,ford,focus se 4dr sedan,fair,4,gas,113193.6,clean,automatic,fwd,unknown,sedan,blue,67.0,"easttexas, TX"


In [99]:
# make too broad
rd_loc = rd_loc.drop(['make'], axis=1)
rd_loc.head()

,price,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,weather,location
0,11900,2010.0,dodge,good,6,gas,43600.0,clean,automatic,rwd,unknown,coupe,red,38.0,"marshall, SD"
1,17550,2008.0,ford,unknown,6,gas,113193.6,clean,automatic,unknown,unknown,unknown,unknown,38.0,"marshall, SD"
3,9900,2007.0,gmc,good,8,gas,169000.0,clean,automatic,4wd,unknown,unknown,unknown,38.0,"marshall, SD"
4,12500,2015.0,jeep,like new,4,gas,39500.0,rebuilt,automatic,4wd,sub-compact,SUV,grey,38.0,"marshall, SD"
6,1250,2004.0,ford,fair,4,gas,113193.6,clean,automatic,fwd,unknown,sedan,blue,67.0,"easttexas, TX"


# Subset the brands?

In [73]:
# 43 brands, keep only the top 15 for now
mfg_list = (rd['manufacturer'].value_counts().head(15)).index.tolist()
mfg_list

['ford',
 'chevrolet',
 'toyota',
 'honda',
 'nissan',
 'dodge',
 'jeep',
 'gmc',
 'ram',
 'bmw',
 'volkswagen',
 'hyundai',
 'chrysler',
 'mercedes-benz',
 'subaru']

# Let's try some groupby

In [93]:
# This gives only the numerical columns. Should I one hot encode the cat columns? Could automatically give me the proportions
rd_loc.groupby(['location']).mean().round(2).squeeze()

,price,year,odometer,weather
location,,,,
"abilene, TX",14732.12,2004.48,121140.24,67.0
"akroncanton, OH",47837.46,2005.14,114005.80,49.0
"albany, MA",11397.68,2006.42,95270.76,50.0
"albany, NY",75564.48,2003.76,115786.44,53.0
"albany, VT",10361.04,1998.26,109755.61,44.0
...,...,...,...,...
"youngstown, PA",6650.55,2000.36,117958.11,53.0
"yubasutter, CA",37907.48,2006.99,116101.87,59.0
"yuma, AZ",8246.79,2002.27,114340.74,71.0


## One hot encoding the cat columns to get proportions by mean

In [107]:
# First want to group the manufacturers I don't want as other
rd_loc.loc[np.logical_not(rd_loc['manufacturer'].isin(mfg_list)), 'manufacturer'] = 'other'
rd_loc['manufacturer'].value_counts()

ford             277467
chevrolet        248224
other            243180
toyota           110824
honda             84493
nissan            75491
dodge             70659
jeep              70507
gmc               65539
ram               52031
bmw               40016
volkswagen        34849
hyundai           30428
chrysler          29752
mercedes-benz     28386
subaru            27693
Name: manufacturer, dtype: int64

In [108]:
rd_loc.columns

Index(['price', 'year', 'manufacturer', 'condition', 'cylinders', 'fuel',
       'odometer', 'title_status', 'transmission', 'drive', 'size', 'type',
       'paint_color', 'weather', 'location'],
      dtype='object')

In [109]:
from sklearn.preprocessing import OneHotEncoder
oh = OneHotEncoder(handle_unknown='ignore', sparse=False)
cat_col = ['manufacturer','condition','cylinders','fuel','title_status','transmission','drive','size','type','paint_color']

In [112]:
OH_cols = pd.DataFrame(oh.fit_transform(rd_loc[cat_col]))
# One-hot encoding removed index; put it back
OH_cols.index = rd_loc.index
OH_cols.columns = oh.get_feature_names(cat_col)
rest = rd_loc.drop(cat_col, axis=1)
OH_rd_loc = pd.concat([rest, OH_cols], axis=1)
OH_rd_loc.head()

,price,year,odometer,weather,location,manufacturer_bmw,manufacturer_chevrolet,manufacturer_chrysler,manufacturer_dodge,manufacturer_ford,...,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_unknown,paint_color_white,paint_color_yellow
0,11900,2010.0,43600.0,38.0,"marshall, SD",0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,17550,2008.0,113193.6,38.0,"marshall, SD",0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,9900,2007.0,169000.0,38.0,"marshall, SD",0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,12500,2015.0,39500.0,38.0,"marshall, SD",0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1250,2004.0,113193.6,67.0,"easttexas, TX",0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [113]:
# Now let's groupby
OH_rd_loc.groupby(['location']).mean().round(2).squeeze()

,price,year,odometer,weather,manufacturer_bmw,manufacturer_chevrolet,manufacturer_chrysler,manufacturer_dodge,manufacturer_ford,manufacturer_gmc,...,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_unknown,paint_color_white,paint_color_yellow
location,,,,,,,,,,,,,,,,,,,,,
"abilene, TX",14732.12,2004.48,121140.24,67.0,0.01,0.24,0.01,0.05,0.26,0.06,...,0.01,0.02,0.05,0.01,0.0,0.08,0.06,0.39,0.19,0.00
"akroncanton, OH",47837.46,2005.14,114005.80,49.0,0.02,0.17,0.03,0.05,0.21,0.04,...,0.01,0.02,0.05,0.00,0.0,0.08,0.09,0.42,0.11,0.01
"albany, MA",11397.68,2006.42,95270.76,50.0,0.00,0.15,0.02,0.06,0.35,0.04,...,0.00,0.04,0.09,0.00,0.0,0.03,0.10,0.21,0.15,0.02
"albany, NY",75564.48,2003.76,115786.44,53.0,0.04,0.16,0.02,0.04,0.15,0.04,...,0.01,0.05,0.08,0.00,0.0,0.07,0.08,0.38,0.10,0.01
"albany, VT",10361.04,1998.26,109755.61,44.0,0.04,0.12,0.01,0.05,0.23,0.04,...,0.04,0.04,0.06,0.01,0.0,0.08,0.09,0.32,0.12,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"youngstown, PA",6650.55,2000.36,117958.11,53.0,0.01,0.25,0.02,0.08,0.20,0.04,...,0.03,0.05,0.03,0.01,0.0,0.09,0.07,0.48,0.09,0.01
"yubasutter, CA",37907.48,2006.99,116101.87,59.0,0.01,0.14,0.03,0.07,0.16,0.05,...,0.00,0.01,0.04,0.00,0.0,0.03,0.05,0.65,0.10,0.00
"yuma, AZ",8246.79,2002.27,114340.74,71.0,0.03,0.17,0.03,0.04,0.20,0.03,...,0.02,0.02,0.07,0.01,0.0,0.06,0.09,0.38,0.16,0.01


# Save final features

In [115]:
features = OH_rd_loc.groupby(['location']).mean().round(2).squeeze()

In [116]:
export = features.to_csv('./data/final_features.csv', header=True, index = True)